In [10]:
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
df = pd.read_csv("data\\train.csv")
sid = SentimentIntensityAnalyzer()

df["keyword"] = df["keyword"].fillna("")
df['score']=df["text"].apply(lambda text:sid.polarity_scores(text)['compound'])
df['keyword'] =df['keyword'].apply(lambda text:sid.polarity_scores(text)['compound'])

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
dtm = tfidf.fit_transform(df['text']).toarray()


In [12]:
import numpy as np
X = pd.DataFrame(dtm)
X[X.shape[1]] = df['keyword']
X[X.shape[1]] = df['score']
X = X.values
y = df['target']

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Concatenate, Input
from tensorflow.keras.models import Model

model = Sequential([
    Dense(128, activation='relu', input_shape=(X.shape[1],)),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # Assuming binary classification
])

# Compile and train the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['f1_score','accuracy'])
model.fit(X, y, epochs=10, batch_size=32, )

C:\Users\prems\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
238/238 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6870 - f1_score: 0.5992 - loss: 0.5924
Epoch 2/10
238/238 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8804 - f1_score: 0.6039 - loss: 0.2978
Epoch 3/10
238/238 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9334 - f1_score: 0.5979 - loss: 0.1917
Epoch 4/10
238/238 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9521 - f1_score: 0.5953 - loss: 0.1330
Epoch 5/10
238/238 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9654 - f1_score: 0.6014 - loss: 0.0935
Epoch 6/10
238/238 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9746 - f1_score: 0.6007 - loss: 0.0621
Epoch 7/10
238/238 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9778 - f1_score: 0.6025 - loss: 0.0538
Epoch 8/10
238/238 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9792 - f1_score: 0.5996 - loss: 0.0475
Epoch 9/10
238/238 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9786 - f1_score: 0.5944 - loss: 0.0440
Epoch 10/10
238/238 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - 

In [14]:
df_test = pd.read_csv("data\\test.csv")
df_test["keyword"] = df_test["keyword"].fillna("")
df_test['score']=df_test["text"].apply(lambda text:sid.polarity_scores(text)['compound'])
df_test['keyword'] =df_test['keyword'].apply(lambda text:sid.polarity_scores(text)['compound'])
test_dtm = tfidf.transform(df_test['text']).toarray() 

X_test = pd.DataFrame(test_dtm)
X_test[X_test.shape[1]] = df_test['keyword']
X_test[X_test.shape[1]] = df_test['score']
X_test = X_test.values

In [15]:
predictions = model.predict(X_test)

102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [16]:
predictions[predictions>0.5]=1
predictions[predictions<=0.5]=0
predictions = predictions.astype(int)

In [17]:
ans = pd.DataFrame(predictions)
ans.rename(columns={0: 'target'}, inplace=True)
ans.insert(0,"id",df_test['id'])
ans.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1


In [18]:
ans.to_csv('output_file.csv', index=False)